In [1]:
import cv2
import numpy as np
import glob

chessboard_size = (9, 9)
square_size = 15

objp = np.zeros((np.prod(chessboard_size), 3), np.float32)
objp[:, :2] = np.indices(chessboard_size).T.reshape(-1, 2)
objp *= square_size

objpoints = []  
imgpoints = []  

images = glob.glob('C:/Users/Kartikay/OneDrive/Desktop/cv/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)
    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)
        cv2.drawChessboardCorners(img, chessboard_size, corners, ret)
        cv2.imshow('Corners', img)
        cv2.waitKey(200)

cv2.destroyAllWindows()

# Calibrate
ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None
)

print("Camera Matrix:\n", camera_matrix)
print("Distortion Coefficients:\n", dist_coeffs)
print("Rotation Vectors:\n", rvecs)
print("Translation Vectors:\n", tvecs)
np.savez("camera_calibration.npz", camera_matrix=camera_matrix, dist_coeffs=dist_coeffs)


Camera Matrix:
 [[883.73992911   0.         641.18955503]
 [  0.         885.34752426 358.54098344]
 [  0.           0.           1.        ]]
Distortion Coefficients:
 [[-1.51204295e-01  5.34699613e-01 -5.89402658e-04 -2.76308300e-03
  -6.25634675e-01]]
Rotation Vectors:
 (array([[0.02371611],
       [0.02633254],
       [1.51907576]]), array([[-0.17740202],
       [ 0.17457261],
       [-1.58928715]]), array([[-0.00590456],
       [-0.06784546],
       [-0.05971344]]), array([[ 0.18194089],
       [-0.18412719],
       [ 1.50551861]]), array([[ 0.1401996 ],
       [-0.15132872],
       [ 1.50864584]]), array([[ 0.12610201],
       [-0.05183966],
       [ 1.53772313]]), array([[-0.1360628 ],
       [-0.36849549],
       [ 1.43813045]]), array([[ 0.127299  ],
       [ 0.14013056],
       [-1.61837499]]), array([[ 0.17272135],
       [ 0.0850262 ],
       [-1.63730198]]))
Translation Vectors:
 (array([[138.91893426],
       [  3.11669668],
       [373.62986955]]), array([[ 75.4639909 ],

In [ ]:
import cv2
import numpy as np

# Load calibration data
with np.load('camera_calibration.npz') as X:
    mtx, dist = [X[i] for i in ('camera_matrix', 'dist_coeffs')]

# Define chessboard size
chessboard_size = (9, 6)
square_size = 25  # same as calibration

# Prepare 3D object points
objp = np.zeros((np.prod(chessboard_size), 3), np.float32)
objp[:, :2] = np.indices(chessboard_size).T.reshape(-1, 2)
objp *= square_size

# Define 3D cube points (size = 3 squares)
cube_size = 3 * square_size
axis = np.float32([
    [0, 0, 0],
    [0, cube_size, 0],
    [cube_size, cube_size, 0],
    [cube_size, 0, 0],
    [0, 0, -cube_size],
    [0, cube_size, -cube_size],
    [cube_size, cube_size, -cube_size],
    [cube_size, 0, -cube_size]
])

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    if ret:
        # Refine corner locations
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1),
            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))

        # Solve PnP → rotation & translation vectors
        ret, rvecs, tvecs = cv2.solvePnP(objp, corners2, mtx, dist)

        # Project 3D cube points to 2D image plane
        imgpts, _ = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)

        imgpts = np.int32(imgpts).reshape(-1, 2)

        # Draw cube
        def draw_cube(img, imgpts):
            img = cv2.drawContours(img, [imgpts[:4]], -1, (0,255,0), 3)
            for i, j in zip(range(4), range(4,8)):
                img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]), (255,0,0), 3)
            img = cv2.drawContours(img, [imgpts[4:]], -1, (0,0,255), 3)
            return img

        frame = draw_cube(frame, imgpts)
    else:
        if rvecs is not None and tvecs is not None:
            rvecs, tvecs = rvecs, tvecs
            ret = True

    cv2.imshow('AR Overlay', frame)
    if cv2.waitKey(1) == 27:  # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()


In [7]:
import cv2
import numpy as np
import math

with np.load('camera_calibration.npz') as X:
    mtx, dist = [X[i] for i in ('camera_matrix', 'dist_coeffs')]

chessboard_size = (9, 6)
square_size = 15

objp = np.zeros((np.prod(chessboard_size), 3), np.float32)
objp[:, :2] = np.indices(chessboard_size).T.reshape(-1, 2)
objp *= square_size

cube_size = 3 * square_size
axis = np.float32([
    [0, 0, 0],
    [0, cube_size, 0],
    [cube_size, cube_size, 0],
    [cube_size, 0, 0],
    [0, 0, -cube_size],
    [0, cube_size, -cube_size],
    [cube_size, cube_size, -cube_size],
    [cube_size, 0, -cube_size]
])

last_rvec, last_tvec = None, None
smooth_rvec, smooth_tvec = None, None
alpha = 0.2 
lost_frames = 0
max_lost_frames = 6
cap = cv2.VideoCapture(0)
frame_count = 0

def draw_cube(img, imgpts):
    imgpts = np.int32(imgpts).reshape(-1, 2)
    img = cv2.drawContours(img, [imgpts[:4]], -1, (0,255,0), 3)
    for i, j in zip(range(4), range(4,8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]), (255,0,0), 3)
    img = cv2.drawContours(img, [imgpts[4:]], -1, (0,0,255), 3)
    return img

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    found, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    if found:
        corners2 = cv2.cornerSubPix(
            gray, corners, (11,11), (-1,-1),
            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        )

        success, rvecs, tvecs, inliers = cv2.solvePnPRansac(
            objp, corners2, mtx, dist, flags=cv2.SOLVEPNP_ITERATIVE
        )

        if success:
            lost_frames = 0
            last_rvec, last_tvec = rvecs, tvecs
    else:
        lost_frames += 1
        success = False
        if last_rvec is not None and last_tvec is not None and lost_frames < max_lost_frames:
            rvecs, tvecs = last_rvec, last_tvec
            success = True  

    if success:
        if smooth_rvec is None:
            smooth_rvec, smooth_tvec = rvecs, tvecs
        else:
            smooth_rvec = alpha * smooth_rvec + (1 - alpha) * rvecs
            smooth_tvec = alpha * smooth_tvec + (1 - alpha) * tvecs

        # frame_count += 1
        # smooth_tvec[1][0] += 10 * math.sin(frame_count / 30.0)

        imgpts, _ = cv2.projectPoints(axis, smooth_rvec, smooth_tvec, mtx, dist)
        frame = draw_cube(frame, imgpts)

        cv2.putText(frame, "Tracking: OK", (20,40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
    else:
        cv2.putText(frame, "Tracking: LOST", (20,40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

    cv2.imshow('AR Overlay', frame)
    if cv2.waitKey(1) == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import math

with np.load('camera_calibration.npz') as X:
    mtx, dist = [X[i] for i in ('camera_matrix', 'dist_coeffs')]

chessboard_size = (9, 6)
square_size = 25

objp = np.zeros((np.prod(chessboard_size), 3), np.float32)
objp[:, :2] = np.indices(chessboard_size).T.reshape(-1, 2)
objp *= square_size

cube_size = 3 * square_size
base_axis = np.float32([
    [0, 0, 0],
    [0, cube_size, 0],
    [cube_size, cube_size, 0],
    [cube_size, 0, 0],
    [0, 0, -cube_size],
    [0, cube_size, -cube_size],
    [cube_size, cube_size, -cube_size],
    [cube_size, 0, -cube_size]
])

last_rvec, last_tvec = None, None
smooth_rvec, smooth_tvec = None, None
alpha = 0.8
lost_frames = 0
max_lost_frames = 10
frame_count = 0

cube_offset = np.array([[0.0, 0.0, 0.0]]) 
cube_rotation = 0.0 

cap = cv2.VideoCapture(0)

def draw_cube(img, imgpts):
    imgpts = np.int32(imgpts).reshape(-1, 2)
    img = cv2.drawContours(img, [imgpts[:4]], -1, (0,255,0), 3)
    for i, j in zip(range(4), range(4,8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]), (255,0,0), 3)
    img = cv2.drawContours(img, [imgpts[4:]], -1, (0,0,255), 3)
    return img

def rotate_y(points, angle_rad):
    R = np.array([
        [ math.cos(angle_rad), 0, math.sin(angle_rad)],
        [ 0, 1, 0],
        [-math.sin(angle_rad), 0, math.cos(angle_rad)]
    ])
    return np.dot(points, R.T)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    found, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    if found:
        corners2 = cv2.cornerSubPix(
            gray, corners, (11,11), (-1,-1),
            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        )
        success, rvecs, tvecs, inliers = cv2.solvePnPRansac(
            objp, corners2, mtx, dist, flags=cv2.SOLVEPNP_ITERATIVE
        )
        if success:
            last_rvec, last_tvec = rvecs, tvecs
            lost_frames = 0
    else:
        lost_frames += 1
        success = False
        if last_rvec is not None and last_tvec is not None and lost_frames < max_lost_frames:
            rvecs, tvecs = last_rvec, last_tvec
            success = True

    if success:
        if smooth_rvec is None:
            smooth_rvec, smooth_tvec = rvecs, tvecs
        else:
            smooth_rvec = alpha * smooth_rvec + (1 - alpha) * rvecs
            smooth_tvec = alpha * smooth_tvec + (1 - alpha) * tvecs

        # frame_count += 1
        # hover = 10 * math.sin(frame_count / 30.0)
        # smooth_tvec[1][0] += hover

        moved_axis = rotate_y(base_axis, cube_rotation) + cube_offset

        imgpts, _ = cv2.projectPoints(moved_axis, smooth_rvec, smooth_tvec, mtx, dist)
        frame = draw_cube(frame, imgpts)
        cv2.putText(frame, "Tracking: OK", (20,40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
    else:
        cv2.putText(frame, "Tracking: LOST", (20,40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

    # Show controls
    cv2.putText(frame, "Controls: W/S-Fwd/Back | A/D-Left/Right | Q/E-Up/Down | R/F-Rotate", 
                (10, frame.shape[0]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

    cv2.imshow('AR Overlay', frame)

    key = cv2.waitKey(10) & 0xFF
    if key == 27:  # ESC
        break
    elif key == ord('w'):
        cube_offset[0][2] -= 10
    elif key == ord('s'):
        cube_offset[0][2] += 10
    elif key == ord('a'):
        cube_offset[0][0] -= 10
    elif key == ord('d'):
        cube_offset[0][0] += 10
    elif key == ord('q'):
        cube_offset[0][1] -= 10
    elif key == ord('e'):
        cube_offset[0][1] += 10

cap.release()
cv2.destroyAllWindows()
